<a href="https://colab.research.google.com/github/angelamyungjoosong/study_data_analytics/blob/main/codes/NLP/healthapp_review_LDA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://heytech.tistory.com/401

In [ ]:
!pip install python-mecab-ko pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.

In [52]:
import numpy as np

In [53]:
# negative 파일 읽기
import pandas as pd
df_review_negative = pd.read_csv('./review_negative.csv')
df_review_negative[:2]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1072,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무 힘들다,1,NaN,NaN
1,1167,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,훌륭,1,NaN,NaN


In [54]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        413 non-null    int64 
 1   app               413 non-null    object
 2   review            413 non-null    object
 3   rating            413 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 19.5+ KB


In [55]:
df_review_negative = df_review_negative.dropna()
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346 entries, 3 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        346 non-null    int64 
 1   app               346 non-null    object
 2   review            346 non-null    object
 3   rating            346 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


In [56]:
# positive 파일 읽기
df_review_positive = pd.read_csv('./review_positive.csv')
df_review_positive[:2]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,NaN,NaN
1,1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고


In [57]:
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        7587 non-null   int64 
 1   app               7587 non-null   object
 2   review            7587 non-null   object
 3   rating            7587 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 355.8+ KB


In [58]:
df_review_positive = df_review_positive.dropna()
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4351 entries, 1 to 7585
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        4351 non-null   int64 
 1   app               4351 non-null   object
 2   review            4351 non-null   object
 3   rating            4351 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 237.9+ KB


# LDA 분석- negative

In [59]:
from gensim.corpora import Dictionary

In [60]:
df_review_negative[['replaced_review']] #pandas에 있는 것 차원배열

,replaced_review
3,칼로리
5,마음
7,오류
8,모야
9,느림
...,...
406,운동 리뷰 출창
407,광고 짜증
408,운동 친절 운동 운동 체계 순서 운동 기분 시작 운동 구간 시작
409,광고 운동


In [61]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [62]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3]) #백터화

[(3, 1)]

In [63]:
# dictionary에 의한 한글 문장을 백터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values:
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [64]:
df_review_negative['문장백터화'] = corpus_list

In [65]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장백터화'],
      dtype='object')

## 토픽 잡기

In [66]:
from gensim.models import LdaModel

In [67]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3 )
#분석할 대상  #index를 word로 바꿔줄 것 #토픽 개수 #이 세가지를 모델에 담음

In [68]:
lda_model.print_topics(num_words=4) #보통 주요단어를 3-5개 정도
#gensim은 ascending이 자동으로 된다
#해당 내용은 페인포인트-> 해당 앱서비스 확인

[(0, '0.025*"운동" + 0.013*"불편" + 0.009*"위젯" + 0.009*"자동 결제 취소"'),
 (1, '0.016*"로그인" + 0.014*"결제 취소" + 0.012*"결제 환불" + 0.009*"결제"'),
 (2, '0.028*"환불" + 0.017*"로그인" + 0.010*"걸음" + 0.010*"최악"')]

# LDA 분석- positive

In [69]:
df_review_positive[['replaced_review']] #pandas에 있는 것 차원배열

,replaced_review
1,최고
4,필요 운동 최고
5,운동 최고
7,집중 운동
12,운동 도움
...,...
7579,리뷰 설명 타임 운동 추천 건강
7580,운동 몸무게 점차 저녁 기대
7582,추천
7583,운동 영상


In [70]:
dictionary = Dictionary(df_review_positive[['replaced_review']].values)
dictionary

In [71]:
dictionary.doc2bow(df_review_positive[['replaced_review']].values[3]) #백터화

[(3, 1)]

In [75]:
# dictionary에 의한 한글 문장을 백터화 변환
corpus_list_positive = list()
for sentence in df_review_positive[['replaced_review']].values:
  vectors = dictionary.doc2bow(sentence)
  corpus_list_positive.append(vectors)

In [76]:
df_review_positive['문장백터화'] = corpus_list_positive

In [77]:
df_review_positive.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장백터화'],
      dtype='object')

## 토픽 잡기

In [78]:
lda_model = LdaModel(corpus=corpus_list_positive, id2word=dictionary, num_topics=3 )

In [79]:
lda_model.print_topics(num_words=4)

[(0, '0.032*"추천" + 0.031*"감사" + 0.014*"운동 도움" + 0.011*"도전"'),
 (1, '0.262*"운동" + 0.027*"도움" + 0.022*"광고" + 0.016*"운동 최고"'),
 (2, '0.082*"최고" + 0.032*"뱃살" + 0.011*"유용" + 0.009*"설명"')]